In [1]:
# install requirements

!pip install torch torchvision --extra-index-url https://download.pytorch.org/whl/cu113


Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu113


In [2]:
# install requirements

!pip install torch-geometric==2.2.0 torch-sparse==0.6.16 torch-scatter==2.1.0 -f https://data.pyg.org/whl/torch-1.12.0+cu113.html

Looking in links: https://data.pyg.org/whl/torch-1.12.0+cu113.html


In [3]:
#Connecting the drive with Colab

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# Making sure the working directory is the one

%cd /content/drive/MyDrive/AGILE

/content/drive/MyDrive/AGILE


In [5]:
import os

In [6]:
# install requirements

!pip install -r requirements.txt